In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.functional import F

MAX_PACKET = 1514.0
BATCH_SIZE = 32
MAX_LENGTH = 50

In [2]:
size_trace = pd.read_csv('flows_size_trace.csv')
time_trace = pd.read_csv('flows_time_trace.csv')
size_trace = size_trace / MAX_PACKET
time_trace = time_trace.divide(time_trace.max(axis=1), axis=0)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
from torch.utils.data import DataLoader, Dataset

class TraceDataset(Dataset):
    def __init__(self, size_df, time_df, window_size=1):
        self.size_df = size_df
        self.time_df = time_df
        self.window_size = window_size
    
    def __len__(self):
        return len(self.size_df.values) - self.window_size
    
    def __getitem__(self, idx):
        s = torch.Tensor(self.size_df.values[idx:idx + self.window_size][:MAX_LENGTH].squeeze())
        t = torch.Tensor(self.time_df.values[idx:idx + self.window_size][:MAX_LENGTH].squeeze())
        target = torch.Tensor([np.sum(self.size_df.values[idx + self.window_size])])
        return s, t, target

trace_train_dataset = TraceDataset(size_trace, time_trace)
trace_train_dataloader = DataLoader(trace_train_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
d1 = 9
d2 = 7
d3 = 4
d4 = 3
d5 = 1
class AEModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.input_size = input_size
        self.slinear1 = nn.Linear(self.input_size, d1)
        self.slinear2 = nn.Linear(d1, d2)
        self.slinear3 = nn.Linear(d2, d3)
        self.slinear4 = nn.Linear(d3, d4)
        self.slinear5 = nn.Linear(d4, d5)
        
        self.tlinear1 = nn.Linear(self.input_size, d1)
        self.tlinear2 = nn.Linear(d1, d2)
        self.tlinear3 = nn.Linear(d2, d3)
        self.tlinear4 = nn.Linear(d3, d4)
        self.tlinear5 = nn.Linear(d4, d5)
        
        self.flinear = nn.Linear(d5 * 2, 1)
        
    def forward(self, s, t):
        rt = t
        rt = self.tlinear1(rt)
        rt = F.relu(rt)
        rt = self.tlinear2(rt)
        rt = F.relu(rt)
        rt = self.tlinear3(rt)
        rt = F.relu(rt)
        rt = self.tlinear4(rt)
        rt = F.relu(rt)
        rt = self.tlinear5(rt)
        rt = F.relu(rt)
        
        rs = s
        rs = self.slinear1(rs)
        rs = F.relu(rs)
        rs = self.slinear2(rs)
        rs = F.relu(rs)
        rs = self.slinear3(rs)
        rs = F.relu(rs)
        rs = self.slinear4(rs)
        rs = F.relu(rs)
        rs = self.slinear5(rs)
        rs = F.relu(rs)
        
        f = torch.concat((rt, rs), dim=1)
        f = self.flinear(f)
        f = F.relu(f)
        
        return f

model = AEModel(len(size_trace.columns)).to(device)
criterion = nn.MSELoss()

In [6]:
learning_rate = 1e-3

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [7]:
NUM_EPOCHS = 20
model.train()
for epoch in range(NUM_EPOCHS):
    for data in trace_train_dataloader:
        s, t, target = data
        s, t, target = s.to(device), t.to(device), target.to(device)
        # ===================forward=====================
        output = model(s, t)
        loss = criterion(output, target)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print(f'epoch [{epoch + 1}/{NUM_EPOCHS}], loss:{loss.item():.4f}')

epoch [1/20], loss:0.5048
epoch [2/20], loss:0.5047
epoch [3/20], loss:0.5047
epoch [4/20], loss:0.5047


KeyboardInterrupt: 